In [1]:
# Libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

from functions import is_promoted

# Warning silencer
pd.options.mode.chained_assignment = None

In [2]:
sales = pd.read_csv("assignment4.1a.csv")
promo_dates = pd.read_csv("PromotionDates.csv")
test = pd.read_csv("assignment4.1b.csv")
categories = pd.read_csv("assignment4.1c.csv")

In [3]:
sales

,Date,StoreCode,ProductCode,SalesQuantity
0,2015-01-01,8,9,-1
1,2015-01-01,131,9,1
2,2015-01-01,144,9,2
3,2015-01-01,203,9,2
4,2015-01-01,256,9,0
...,...,...,...,...
1873613,2015-07-30,292,315,0
1873614,2015-07-31,12,315,1
1873615,2015-07-31,104,315,1
1873616,2015-07-31,261,315,1


In [4]:
test

,Date,StoreCode,ProductCode,SalesQuantity
0,2015-08-01,2,9,1
1,2015-08-01,7,9,1
2,2015-08-01,62,9,1
3,2015-08-01,181,9,1
4,2015-08-01,6,20,1
...,...,...,...,...
1028116,2016-01-01,340,247,5
1028117,2016-01-01,340,264,1
1028118,2016-01-01,340,292,1
1028119,2016-01-01,340,305,1


In [5]:
round(sales["SalesQuantity"].describe(), 3)

count    1873618.000
mean           2.247
std            5.029
min          -60.000
25%            0.000
50%            1.000
75%            2.000
max          912.000
Name: SalesQuantity, dtype: float64

In [6]:
sales[["Date", "StoreCode", "ProductCode"]].shape == sales[["Date", "StoreCode", "ProductCode"]].drop_duplicates().shape

True

In [7]:
print("# of Products: {}".format(sales["ProductCode"].unique().size))
print("# of Stores: {}".format(sales["StoreCode"].unique().size))

# of Products: 317
# of Stores: 340


In [8]:
promo_dates

,Period,StartDate,EndDate
0,Promo1,2/10/2015,2/17/2015
1,Promo2,3/15/2015,3/22/2015
2,Promo3,5/24/2015,6/1/2015
3,Promo4,6/21/2015,6/28/2015
4,Promo5,1/9/2015,6/9/2015
5,Promo6,20/11/2015,27/11/2015


In [9]:
promo_dates = promo_dates.iloc[:5,:]    # Drop the last row

# Only first 4 promotions:
promo_dates_test = promo_dates[promo_dates["Period"] == "Promo5"]
promo_dates_test["StartDate"] = pd.to_datetime(promo_dates_test["StartDate"], format="%d/%m/%Y")
promo_dates_test["EndDate"] = pd.to_datetime(promo_dates_test["EndDate"], format="%d/%m/%Y")
promo_dates = promo_dates.iloc[:4,:]
promo_dates["StartDate"] = pd.to_datetime(promo_dates["StartDate"], format="%m/%d/%Y")
promo_dates["EndDate"] = pd.to_datetime(promo_dates["EndDate"], format="%m/%d/%Y")

In [10]:
# Data type adjustments for date columns
sales["Date"] = pd.to_datetime(sales["Date"])
test["Date"] = pd.to_datetime(test["Date"])
# Storing promo dates as array to iterate over
promo_dates = np.array(promo_dates.drop("Period", axis=1))
promo_dates_test = np.array(promo_dates_test.drop("Period", axis=1))

In [12]:
# Training data
sales["IsPromoted"] = sales["Date"].apply(is_promoted, promo_dates=promo_dates) # Promotions as a binary data column
test["IsPromoted"] = test["Date"].apply(is_promoted, promo_dates=promo_dates_test)
train = sales.merge(categories, how='left', on='ProductCode')                   # Adding the category information
test = test.merge(categories, how='left', on='ProductCode')
train

,Date,StoreCode,ProductCode,SalesQuantity,IsPromoted,ProductGroup1,ProductGroup2
0,2015-01-01,8,9,-1,False,H,15
1,2015-01-01,131,9,1,False,H,15
2,2015-01-01,144,9,2,False,H,15
3,2015-01-01,203,9,2,False,H,15
4,2015-01-01,256,9,0,False,H,15
...,...,...,...,...,...,...,...
1873613,2015-07-30,292,315,0,False,D,16
1873614,2015-07-31,12,315,1,False,D,16
1873615,2015-07-31,104,315,1,False,D,16
1873616,2015-07-31,261,315,1,False,D,16


In [13]:
test

,Date,StoreCode,ProductCode,SalesQuantity,IsPromoted,ProductGroup1,ProductGroup2
0,2015-08-01,2,9,1,False,H,15
1,2015-08-01,7,9,1,False,H,15
2,2015-08-01,62,9,1,False,H,15
3,2015-08-01,181,9,1,False,H,15
4,2015-08-01,6,20,1,False,A,6
...,...,...,...,...,...,...,...
1028116,2016-01-01,340,247,5,False,A,5
1028117,2016-01-01,340,264,1,False,A,5
1028118,2016-01-01,340,292,1,False,I,10
1028119,2016-01-01,340,305,1,False,G,3


In [11]:
train.to_pickle("train.pkl")

In [14]:
test.to_pickle("test.pkl")